# Fragile Families Project

In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn_pandas import DataFrameMapper

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
######## VARIABLE OF INTEREST ########
test_var = 'jobTraining'

## Preprocessing

Data was first imputed naively by removing columns that were completely NA, then filling in missing data with modes

In [3]:
features = pd.read_csv('background_imputed_3.csv', low_memory=False, index_col='challengeID')
print("Features shape: {}".format(features.shape))
features.sort_index(kind='mergesort', inplace=True) # mergesort is the only stable sort
features.head()

Features shape: (4242, 12007)


,# idnum,mothid1,m1intmon,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,...,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
1.0,1854.0,18540.0,1.0,0.0,40.0,0.0,1.0,202.4850,1.0,1.0,...,12.1322,10.99190,10.56410,1.0,10.97270,9.56668,7986.0,189.0,0.0,0.0
2.0,4470.0,44700.0,1.0,0.0,40.0,1.0,1.0,45.6082,1.0,1.0,...,12.1322,10.99190,9.84507,1.0,10.97270,9.72355,3280.0,2552.0,24.0,15.0
3.0,371.0,3710.0,1.0,0.0,35.0,1.0,1.0,39.0603,1.0,1.0,...,12.1322,10.99190,10.56410,1.0,10.97270,9.56668,4597.0,1925.0,45.0,0.0
4.0,2654.0,26540.0,1.0,0.0,30.0,1.0,1.0,22.3049,1.0,1.0,...,12.1322,10.67730,10.56410,1.0,8.52233,10.60810,263.0,4494.0,12.0,1.0
5.0,2810.0,28100.0,1.0,0.0,25.0,1.0,1.0,35.5183,1.0,1.0,...,12.1322,9.73198,10.56410,1.0,10.11530,9.64647,1005.0,4067.0,29.0,0.0


In [4]:
# holders for later on...

n = len(features)+1
pred_gpa_y = np.full(n-1, 2.866)
pred_grit_y = np.full(n-1, 3.427)
pred_material_y = np.full(n-1, 0.0596)
pred_evict_y = np.full(n-1, 0.0596)
pred_layoff_y = np.full(n-1, 0.209)
pred_job_y = np.full(n-1, 0.334)

In [5]:
# some columns are not numeric; encode them
from sklearn.preprocessing import LabelEncoder
str_col_features = features.select_dtypes(include=['object'])
label_encoders={} # save the label encoders to allow for inverse transforms
for col_name in str_col_features.columns:
    le = LabelEncoder()
    dmy = le.fit(features[col_name])
    features[col_name] = le.transform(features[col_name])
    label_encoders[col_name] = le
features.head()

,# idnum,mothid1,m1intmon,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,...,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
1.0,1854.0,18540.0,1.0,0.0,40.0,0.0,1.0,202.4850,1.0,1.0,...,12.1322,10.99190,10.56410,1.0,10.97270,9.56668,7986.0,189.0,0.0,0.0
2.0,4470.0,44700.0,1.0,0.0,40.0,1.0,1.0,45.6082,1.0,1.0,...,12.1322,10.99190,9.84507,1.0,10.97270,9.72355,3280.0,2552.0,24.0,15.0
3.0,371.0,3710.0,1.0,0.0,35.0,1.0,1.0,39.0603,1.0,1.0,...,12.1322,10.99190,10.56410,1.0,10.97270,9.56668,4597.0,1925.0,45.0,0.0
4.0,2654.0,26540.0,1.0,0.0,30.0,1.0,1.0,22.3049,1.0,1.0,...,12.1322,10.67730,10.56410,1.0,8.52233,10.60810,263.0,4494.0,12.0,1.0
5.0,2810.0,28100.0,1.0,0.0,25.0,1.0,1.0,35.5183,1.0,1.0,...,12.1322,9.73198,10.56410,1.0,10.11530,9.64647,1005.0,4067.0,29.0,0.0


In [6]:
# scale data to unit variance and 0 mean
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
indices_f = features.index
colnames_f = features.columns
features = ss.fit_transform(features)
features = pd.DataFrame(data=features, index=indices_f, columns=colnames_f)
features.head()

,# idnum,mothid1,m1intmon,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,...,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
1.0,-0.416725,-0.416725,0.0,-0.30666,0.385580,-1.880664,0.162398,0.702597,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,-0.586254,2.694875,-2.617362,-1.333624,-0.548459
2.0,1.439058,1.439058,0.0,-0.30666,0.385580,0.531727,0.162398,-0.184623,0.601133,0.162398,...,0.340443,0.392404,-1.256213,0.0,0.401826,-0.273756,0.515323,-0.611222,-0.602345,0.425379
3.0,-1.468761,-1.468761,0.0,-0.30666,-0.051404,0.531727,0.162398,-0.221655,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,-0.586254,1.125283,-1.143532,0.037524,-0.548459
4.0,0.150793,0.150793,0.0,-0.30666,-0.488388,0.531727,0.162398,-0.316416,0.601133,0.162398,...,0.340443,-0.040350,0.307392,0.0,-3.309481,1.488338,-0.881979,1.037498,-0.967985,-0.483537
5.0,0.261459,0.261459,0.0,-0.30666,-0.925373,0.531727,0.162398,-0.241687,0.601133,0.162398,...,0.340443,-1.340702,0.307392,0.0,-0.896784,-0.427306,-0.538327,0.674983,-0.449996,-0.548459


In [7]:
from sklearn.model_selection import train_test_split

full_y = pd.read_csv('train.csv', index_col='challengeID')
train_y = full_y[test_var] # variable we are interested in
print("Target shape: {}".format(train_y.shape))
train_y.dropna(how='any', inplace=True) # drop those with no reported outcome
print("Target shape (no na): {}".format(train_y.shape))

# get the rows for which we can predict
y_indices = train_y.index.values.tolist()
full_x = features
train_x = features.loc[y_indices]
print("Features shape (final): {}".format(train_x.shape))

# train_x.to_csv("full_x.csv", index=True)
# train_y.to_csv("full_y.csv", index=True)

train_x, test_x, train_y, test_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=0)

train_x.shape
train_x.head()
train_y.shape
train_y.head()
test_x.shape
test_x.head()
test_y.shape
test_y.head()

Target shape: (2121,)
Target shape (no na): (1461,)
Features shape (final): (1461, 12007)


(1168, 12007)

,# idnum,mothid1,m1intmon,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,...,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
1814,0.730370,0.730370,0.0,-0.306660,-0.051404,-1.880664,0.162398,0.046873,0.601133,0.162398,...,0.340443,0.392404,-0.733808,0.0,0.401826,-0.586254,0.770978,-0.860822,-1.059395,-0.418614
266,-1.055183,-1.055183,0.0,-0.306660,1.783930,-1.880664,0.162398,0.023866,-1.663526,0.162398,...,0.340443,-2.541036,0.307392,0.0,-3.466408,-0.586254,-0.855117,1.206445,-0.724225,-0.094001
2366,1.224111,1.224111,0.0,2.490869,-0.488388,0.531727,0.162398,-0.303517,-1.663526,0.162398,...,0.340443,0.392404,0.307392,0.0,-1.572746,0.347673,-0.999155,1.210690,-0.267176,1.074604
837,0.930420,0.930420,0.0,-0.306660,-0.488388,0.531727,0.162398,-0.319201,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,0.068324,0.171671,-0.265687,-0.785165,4.385654
3591,-0.556476,-0.556476,0.0,-0.306660,1.259549,0.531727,0.162398,1.784719,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,-2.244663,0.786528,-0.306756,0.359163,0.189873,0.035844


(1168,)

challengeID
1814     True
266     False
2366    False
837      True
3591    False
Name: jobTraining, dtype: object

(293, 12007)

,# idnum,mothid1,m1intmon,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,...,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
1500,0.863737,0.863737,0.0,-0.306660,1.259549,0.531727,0.162398,-0.211812,-1.663526,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,0.786528,-0.615209,0.493302,1.926661,0.035844
1834,0.675747,0.675747,0.0,-0.306660,1.259549,0.531727,0.162398,-0.386257,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,-0.586254,0.077190,0.054379,0.982092,0.944759
1304,-0.440845,-0.440845,0.0,-0.306660,-0.488388,0.531727,0.162398,-0.205090,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,3.852938,0.192049,-0.260593,0.220343,-0.158924
791,0.647371,0.647371,0.0,-0.306660,-0.488388,-1.880664,0.162398,1.503346,0.601133,0.162398,...,0.340443,0.392404,-2.697498,0.0,0.401826,1.794322,0.661213,-0.824316,0.189873,-0.353692
1833,0.570756,0.570756,0.0,2.490869,-3.110294,0.531727,0.162398,5.845136,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,-0.187198,-0.586254,-1.001933,1.125792,-0.724225,-0.223847


(293,)

challengeID
1500    False
1834     True
1304    False
791     False
1833    False
Name: jobTraining, dtype: object

## Feature Selection

First remove features with 0 variance so that real feature selection can be sped up

In [8]:
from sklearn.feature_selection import VarianceThreshold
# remove features with 0 variance

# mapper from pandas to numpy
mapper = DataFrameMapper([(train_x.columns, None)])
indices = train_x.index
colnames = train_x.columns

vt = VarianceThreshold()
train_vt_x = vt.fit_transform(mapper.fit_transform(train_x))
train_x = pd.DataFrame(data=train_vt_x, index=indices, columns=colnames[vt.get_support(True)])
train_x.head()

# Test set needs variance threshold treatment
# mapper from pandas to numpy
mapper_test = DataFrameMapper([(test_x.columns, None)])
indices = test_x.index
colnames = test_x.columns

test_vt_x = vt.transform(mapper.fit_transform(test_x))
test_x = pd.DataFrame(data=test_vt_x, index=indices, columns=colnames[vt.get_support(True)])
test_x.head()

,# idnum,mothid1,m1lenhr,m1lenmin,cm1fint,m1citywt,innatsm,m1a3,m1a4,m1a5,...,f4d7,f4d9,m5c6,m5d20,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
1814,0.730370,0.730370,-0.306660,-0.051404,-1.880664,0.046873,0.601133,-0.312772,-1.754831,-0.424618,...,-0.472812,0.340443,0.392404,-0.733808,0.401826,-0.586254,0.770978,-0.860822,-1.059395,-0.418614
266,-1.055183,-1.055183,-0.306660,1.783930,-1.880664,0.023866,-1.663526,-0.312772,-1.754831,-0.424618,...,2.707464,0.340443,-2.541036,0.307392,-3.466408,-0.586254,-0.855117,1.206445,-0.724225,-0.094001
2366,1.224111,1.224111,2.490869,-0.488388,0.531727,-0.303517,-1.663526,-0.312772,-1.754831,-0.424618,...,1.398283,0.340443,0.392404,0.307392,-1.572746,0.347673,-0.999155,1.210690,-0.267176,1.074604
837,0.930420,0.930420,-0.306660,-0.488388,0.531727,-0.319201,0.601133,-0.312772,0.554100,-0.424618,...,-0.472812,0.340443,0.392404,0.307392,0.401826,0.068324,0.171671,-0.265687,-0.785165,4.385654
3591,-0.556476,-0.556476,-0.306660,1.259549,0.531727,1.784719,0.601133,-0.312772,0.554100,-0.424618,...,-0.472812,0.340443,0.392404,0.307392,-2.244663,0.786528,-0.306756,0.359163,0.189873,0.035844


,# idnum,mothid1,m1lenhr,m1lenmin,cm1fint,m1citywt,innatsm,m1a3,m1a4,m1a5,...,f4d7,f4d9,m5c6,m5d20,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
1500,0.863737,0.863737,-0.306660,1.259549,0.531727,-0.211812,-1.663526,-0.312772,0.554100,-0.424618,...,0.060564,0.340443,0.392404,0.307392,0.401826,0.786528,-0.615209,0.493302,1.926661,0.035844
1834,0.675747,0.675747,-0.306660,1.259549,0.531727,-0.386257,0.601133,-0.312772,0.554100,2.355058,...,-0.472812,0.340443,0.392404,0.307392,0.401826,-0.586254,0.077190,0.054379,0.982092,0.944759
1304,-0.440845,-0.440845,-0.306660,-0.488388,0.531727,-0.205090,0.601133,-0.312772,0.554100,-0.424618,...,-0.168481,0.340443,0.392404,0.307392,0.401826,3.852938,0.192049,-0.260593,0.220343,-0.158924
791,0.647371,0.647371,-0.306660,-0.488388,-1.880664,1.503346,0.601133,-0.312772,0.554100,2.355058,...,-0.472812,0.340443,0.392404,-2.697498,0.401826,1.794322,0.661213,-0.824316,0.189873,-0.353692
1833,0.570756,0.570756,2.490869,-3.110294,0.531727,5.845136,0.601133,-0.312772,-1.754831,-0.424618,...,0.057293,0.340443,0.392404,0.307392,-0.187198,-0.586254,-1.001933,1.125792,-0.724225,-0.223847


Next use some regression model to find most relevant features.  Here we select features based on Lasso and mutual info as well as importance assigned to features based on decision trees.

In [9]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel

from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LassoCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# mapper from pandas to numpy
mapper = DataFrameMapper([(train_x.columns, None)])
indices = train_x.index
colnames = train_x.columns

In [10]:
# Use Lasso Regression
fs_model_lasso = LassoCV().fit(mapper.fit_transform(train_x), train_y.as_matrix())
fs_lasso = SelectFromModel(fs_model_lasso, prefit=True, threshold=0.01)
train_fs_lasso_x = fs_lasso.transform(train_x)
train_fs_lasso_x = pd.DataFrame(data=train_fs_lasso_x, index=indices, columns=colnames[fs_lasso.get_support(True)])
train_fs_lasso_x.head()

scores_lasso = np.column_stack((colnames, fs_model_lasso.coef_))
scores_lasso = scores_lasso[fs_lasso.get_support(True)]
scores_lasso = np.core.records.fromarrays(scores_lasso.transpose(), names='name, score', formats = 'S8, f8')
scores_lasso = np.sort(scores_lasso, order=['score'], kind='mergesort')
print("Lasso # of features: {}".format(len(scores_lasso)))
print(scores_lasso['name'])

np.savetxt("{}_features_lasso.csv".format(test_var), scores_lasso['name'], fmt="%s")

# fig1 = plt.figure(1)
# dmy = fig1.clear()
# y_pos1 = np.arange(len(scores_lasso))
# dmy = plt.title('Lasso')
# dmy = plt.bar(y_pos1, scores_lasso['score'], align='center')
# dmy = plt.xticks(y_pos1, scores_lasso['name'], rotation='vertical')
# fig1

/home/rfong/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


,f2k5a8,m5i1,m5i3b,m5i19a,p5l13f,cm5edu
challengeID,,,,,,
1814,0.034352,0.393991,0.503901,-0.209370,0.292345,0.387317
266,0.034352,0.393991,0.503901,0.235228,0.292345,0.387317
2366,0.034352,0.393991,0.503901,-0.655288,0.292345,-0.759229
837,0.034352,0.393991,-1.984515,-0.655288,-3.420614,-0.759229
3591,0.034352,0.393991,0.503901,-0.655288,0.292345,-1.905775


Lasso # of features: 6
['p5l13f' 'm5i1' 'm5i3b' 'f2k5a8' 'cm5edu' 'm5i19a']


In [11]:
# Decision Tree to select features
# if test_var == 'gpa' or test_var == 'materialHardship': # regressor
fs_model_decision = DecisionTreeRegressor().fit(mapper.fit_transform(train_x), train_y.as_matrix())
# else: # classifier
#     fs_model_decision = DecisionTreeClassifier().fit(mapper.fit_transform(train_x), train_y.as_matrix())
    
fs_decision = SelectFromModel(fs_model_decision, prefit=True, threshold=0.01)
train_fs_decision_x = fs_decision.transform(train_x)
train_fs_decision_x = pd.DataFrame(data=train_fs_decision_x, index=indices, columns=colnames[fs_decision.get_support(True)])
train_fs_decision_x.head()

scores_decision = np.column_stack((colnames, fs_model_decision.feature_importances_))
scores_decision = scores_decision[fs_decision.get_support(True)]
scores_decision = np.core.records.fromarrays(scores_decision.transpose(), names='name, score', formats = 'S8, f8')
scores_decision = np.sort(scores_decision, order=['score'], kind='mergesort')
print("Decision # of features: {}".format(len(scores_decision)))
print(scores_decision['name'])
        

# fig2 = plt.figure(2)
# dmy = fig2.clear()
# y_pos2 = np.arange(len(scores_decision))
# dmy = plt.title('Decision Tree')
# dmy = plt.bar(y_pos2, scores_decision['score'], align='center')
# dmy = plt.xticks(y_pos2, scores_decision['name'], rotation='vertical')
# fig2

,m1f3,m1i2ii,cm1hhinc,m2b5c1,m2c8d,m2c19a,cm2fbir,m2citywt_rep1,f2g2,f2h8c2,...,hv5_mhtcm,hv3e4,hv3pvceilr_m,hv4a27,hv4l9,hv4ppvtraw,hv4pvceilr,hv4hap,kind_attention_scale,ffcc_famsurvey_f30
challengeID,,,,,,,,,,,,,,,,,,,,,
1814,0.345342,0.68985,0.523141,-0.000919,-0.27788,0.377047,1.754249,0.106267,-0.364214,-0.211842,...,1.001541,4.809013,-0.504382,-0.497589,-0.455417,0.058577,0.595504,0.299582,-0.343493,-0.189769
266,0.345342,0.06523,-0.738383,0.687239,-0.27788,0.377047,2.556246,0.113306,2.063117,-0.211842,...,-0.177301,-0.334622,-1.392915,-0.497589,-0.455417,-0.157560,-0.596347,-1.878652,-0.343493,-0.189769
2366,0.345342,-0.55939,-1.023328,0.228467,-0.27788,0.377047,-0.651740,-0.194777,-0.768769,-0.211842,...,0.221120,-0.334622,0.384151,-0.497589,-0.455417,0.058577,0.595504,0.299582,-0.343493,-0.189769
837,0.345342,0.06523,-0.014987,-0.689077,-0.27788,0.377047,1.152752,-0.232647,-0.364214,-0.211842,...,0.174396,-0.334622,-1.392915,-0.497589,-0.455417,0.058577,0.595504,0.299582,-0.343493,-0.189769
3591,0.345342,0.06523,-0.849900,-0.230305,-0.27788,0.377047,-0.451241,-0.362468,-0.364214,-0.211842,...,0.186469,-0.334622,0.384151,2.955616,-0.455417,-0.229605,-0.000421,0.225359,-0.343493,-0.189769


Decision # of features: 43
['p5h9' 'p5i1h' 'ffcc_fam' 'm5j2' 'o5e18h' 'm1i2ii' 'hv4l9' 'f3c30d'
 'hv4pvcei' 'cm2fbir' 'm2b5c1' 'hv4a27' 'm1f3' 'p5q1g' 'm2citywt' 'm2c8d'
 'p5q1j' 'cf3alc_c' 'cf3povco' 'f2h8c2' 'm4c13a1' 'p5h8' 'hv4hap' 'm3k16b'
 'f3i24' 'hv4ppvtr' 'cm1hhinc' 'f5k1' 'm5i3b' 'hv3e4' 'm2c19a' 'p5i1g'
 'kind_att' 'f2g2' 'hv3pvcei' 'm4citywt' 'm5i2_15' 'hv5_mhtc' 'p5l13f'
 'cm4b_age' 'm3a13' 'f4e18g' 'm3b17']


In [12]:
# Use Random forest to select features
# if test_var == 'gpa' or test_var == 'materialHardship': # regressor
fs_model_forest = RandomForestRegressor().fit(mapper.fit_transform(train_x), train_y.as_matrix())
# else:
#     fs_model_forest = RandomForestClassifier().fit(mapper.fit_transform(train_x), train_y.as_matrix())

fs_forest = SelectFromModel(fs_model_forest, prefit=True, threshold=0.005)
train_fs_forest_x = fs_forest.transform(train_x)
train_fs_forest_x = pd.DataFrame(data=train_fs_forest_x, index=indices, columns=colnames[fs_forest.get_support(True)])
train_fs_forest_x.head()

scores_forest = np.column_stack((colnames, fs_model_forest.feature_importances_))
scores_forest = scores_forest[fs_forest.get_support(True)]
scores_forest = np.core.records.fromarrays(scores_forest.transpose(), names='name, score', formats = 'S8, f8')
scores_forest = np.sort(scores_forest, order=['score'], kind='mergesort')
print("Random Forest # of features: {}".format(len(scores_forest)))
print(scores_forest['name'])

# fig3 = plt.figure(3)
# dmy = fig3.clear()
# y_pos3 = np.arange(len(scores_forest))
# dmy = plt.title('Random Forest')
# dmy = plt.bar(y_pos3, scores_forest['score'], align='center')
# dmy = plt.xticks(y_pos3, scores_forest['name'], rotation='vertical')
# fig3

,m1e1d1,m1j2d,m2c3d,f2k23e,m3k19,m3k22,m3l2a,f3j2e2,cf4povco,m5i3b,m5i19a,f5i24e,p5l13f,cf5hhinc,hv4waz,m3d9
challengeID,,,,,,,,,,,,,,,,
1814,0.031334,-1.199786,-0.209563,-0.108406,-0.686608,-0.387635,-0.368376,1.188868,-0.802359,0.503901,-0.209370,-0.150691,0.292345,-0.724845,-0.021550,-0.554629
266,0.223414,-0.249117,3.950030,-0.108406,-0.686608,-1.206941,-0.368376,0.015149,0.664917,0.503901,0.235228,-0.150691,0.292345,0.224151,0.904220,-0.554629
2366,-0.096719,0.627910,3.908434,-0.108406,-0.686608,-0.387635,-0.368376,0.602008,-0.002751,0.503901,-0.655288,1.930786,0.292345,0.724146,-0.021550,-0.554629
837,0.095361,-0.249117,-0.230361,-0.108406,1.371646,-0.387635,1.855439,0.602008,-0.802359,-1.984515,-0.655288,-0.150691,-3.420614,-0.724845,-0.021550,-0.554629
3591,-0.096719,-0.249117,-0.292755,-0.108406,-0.673269,-1.206941,0.855854,-0.571711,-0.570149,0.503901,-0.655288,-0.150691,0.292345,-0.358000,-1.233324,-0.554629


Random Forest # of features: 16
['hv4waz' 'm3k19' 'f5i24e' 'f2k23e' 'm1e1d1' 'm1j2d' 'm5i19a' 'm2c3d'
 'cf5hhinc' 'm3k22' 'cf4povco' 'm3l2a' 'm3d9' 'f3j2e2' 'p5l13f' 'm5i3b']


In [13]:
# Use mutual info
fs_mutualinfo = SelectKBest(mutual_info_regression, k=50)
fs_mutualinfo.fit(mapper.fit_transform(train_x), train_y.as_matrix())
train_fs_mutualinfo_x = fs_mutualinfo.transform(train_x)
train_fs_mutualinfo_x = pd.DataFrame(data=train_fs_mutualinfo_x, index=indices, columns=colnames[fs_mutualinfo.get_support(True)])
train_fs_mutualinfo_x.head()

scores_mutualinfo = np.column_stack((colnames, fs_mutualinfo.scores_))
scores_mutualinfo = scores_mutualinfo[fs_mutualinfo.get_support(True)]
scores_mutualinfo = np.core.records.fromarrays(scores_mutualinfo.transpose(), names='name, score', formats = 'S8, f8')
scores_mutualinfo = np.sort(scores_mutualinfo, order=['score'], kind='mergesort')
print("MI # of features: {}".format(len(scores_mutualinfo)))
print(scores_mutualinfo['name'])

# fig4 = plt.figure(4)
# dmy = fig4.clear()
# y_pos4 = np.arange(len(scores_mutualinfo))
# dmy = plt.title('Mutual Info')
# dmy = plt.bar(y_pos4, scores_mutualinfo['score'], align='center')
# dmy = plt.xticks(y_pos4, scores_mutualinfo['name'], rotation='vertical')

SelectKBest(k=50,
      score_func=<function mutual_info_regression at 0x7ff240f03848>)

,m1j1a,f1b11d,f1e1d6,f1e6a,f1e6c,m2b24a1,m2h16,m2h16a,m2k3a1,f2a4d,...,hv3e4_per,hv3h3c,hv3j18,hv3q2,hv3r9,hv3t7,ffcc_pof_d13,ffcc_centsurvey_c7_a,ffcc_centsurvey_e4_b,ffcc_famsurvey_b13b_3
challengeID,,,,,,,,,,,,,,,,,,,,,
1814,-0.652229,-0.221272,-0.138457,0.470555,0.452895,-0.030722,0.375414,0.209283,0.113552,-0.272093,...,0.157654,-0.812975,-0.116011,-0.040656,-0.305449,0.274878,0.034352,-0.042746,0.202901,-0.169154
266,1.533203,-0.221272,-0.138457,0.470555,0.452895,-0.030722,-2.663723,0.209283,0.113552,-0.272093,...,0.157654,0.546363,-0.116011,-0.040656,-0.305449,0.274878,0.034352,-0.042746,0.202901,-0.169154
2366,1.533203,-0.221272,-0.138457,0.470555,-2.208019,-0.030722,0.375414,0.209283,0.113552,-0.272093,...,0.157654,0.546363,-0.116011,-0.040656,-0.305449,0.274878,0.034352,-0.042746,0.202901,-0.169154
837,-0.652229,-0.221272,-0.138457,0.470555,0.452895,-0.030722,0.375414,0.209283,0.113552,-0.272093,...,0.157654,-0.812975,-0.116011,-0.040656,-0.305449,0.274878,0.034352,-0.042746,0.202901,-0.169154
3591,1.533203,-0.221272,-0.138457,0.470555,0.452895,-0.030722,0.375414,0.209283,0.113552,-0.272093,...,0.157654,0.546363,-0.116011,-0.040656,-0.305449,0.274878,0.034352,-0.042746,0.202901,-0.169154


MI # of features: 50
['hv3r9' 'k5b3b' 'hv3j18' 'f4e14' 'f3b3e_2' 'f1b11d' 'f2a4d' 'm1j1a'
 'm5i2_11' 'm2h16a' 'ffcc_fam' 'f1e6a' 'm3k3a_16' 'f3j2' 'f2k5a16'
 'm2b24a1' 'f3f2e4' 'm3k13p' 'm2h16' 'm3b3e_4' 'm3a4c2' 'm4b26c' 'm3e23f'
 'p5q3at' 'm4f2d2' 'ffcc_cen' 'n5c2c' 'm4f3a1_3' 'hv3t7' 'm2k3a1' 'hv3h3c'
 'f4j25a2' 'm4j25b2' 'f1e1d6' 'hv3b16a_' 'f4j10' 'm5b16a' 'm4k3b'
 'hv3e4_pe' 'hv3d13' 'ffcc_pof' 'f5b21' 'f3b16a3' 'm4c2f' 'f1e6c' 'f5j6a'
 'f2c13c1' 'hv3q2' 'f5e1f' 'ffcc_cen']


## Model Selection and Fitting

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error

# For continuous regressors

# est- the regressor, params - params to CV, transform - to do any feature selection
regressors = [
    {
        'est': DecisionTreeRegressor(),
        'params': {
            "max_depth": [1000, None],
            "max_features": [None],
            "criterion": ["mse", "mae"]
        },
        'transform': [fs_decision, fs_lasso, fs_mutualinfo]
    },
    {
        'est': RandomForestRegressor(),
        'params': {"max_depth": [1000, None],
                  "max_features": [None],
                  "criterion": ["mse", "mae"]},
        'transform': [fs_forest, fs_lasso, fs_mutualinfo]
    },
    {
        'est': Lasso(),
        'params': {
            "alpha": [0.5, 1, 2, 4, 8],
            "fit_intercept": [True, False],
            "normalize": [True, False]
        },
        'transform': [fs_lasso, fs_mutualinfo]
    },
    {
        'est': LinearRegression(),
        'params': {
            "fit_intercept": [True, False],
            "normalize": [True, False]
        },
        'transform': [fs_lasso, fs_mutualinfo]
        
    },
    {
        'est': ElasticNet(),
        'params': {
            "alpha": [0.5, 1, 2, 4, 8],
            "l1_ratio": [0.1, 0.2, 0.4, 0.6, 0.8],
            "fit_intercept": [True, False],
            "normalize": [True, False],
            "positive": [True, False]
        },
        'transform': [fs_lasso, fs_mutualinfo]
    },
    {
        'est': LinearSVR(),
        'params': {
            "C": [1, 2, 4]
        },
        'transform': [fs_lasso, fs_mutualinfo]
        
    }
]

# classifiers for discrete vars
# classifiers = [
#     {
#         'est': MultinomialNB(),
#         'params': {
#             'alpha': [1, 0.1, 0.01, 0]
#                   },
#         'transform': [fs_lasso, fs_mutualinfo]
#     },
#     { 
#         'est': KNeighborsClassifier(),
#         'params': {
#             'n_neighbors': [2, 3, 4, 6, 10, 14, 18]
#         },
#         'transform': [fs_lasso, fs_mutualinfo]
#     },
#     {
#         'est': DecisionTreeClassifier(),
#         'params': {
#             'criterion': ['gini', 'entropy'],
#             'min_samples_split': [2, 4, 6, 8],
#             'max_depth': [1000, None],
#             'min_samples_leaf': [3, 6, 9, 12 ]
#         },
#         'transform': [fs_decision, fs_lasso, fs_mutualinfo]
#     },
#     {
#         'est': LinearSVC(),
#         'params': {
#             'C': [0.5, 1, 3, 6, 8]
#         },
#         'transform': [fs_lasso, fs_mutualinfo]
#     },
#     {
#         'est': RandomForestClassifier(),
#         'params': {
#             'criterion': ['gini', 'entropy'],
#             'max_depth': [1000, None],
#             'n_estimators': [5, 10, 20, 25]
#         },
#         'transform': [fs_forest, fs_lasso, fs_mutualinfo]
#     }
# ]

# if test_var == 'gpa' or test_var == 'materialHardship':
estimators = regressors
# else:
#     estimators = classifiers

# run grid search without feature selection
# keeping track of best classifier
best_mse = 100
best_clf = 0
best_transformer = 0
for est in estimators: 
    model = est['est']
    print("~~~ Fitting and Testing {} ~~~".format(type(model).__name__))
    grid_search = GridSearchCV(model, param_grid=est['params'])
    for transformer in est['transform']:
        "~Transforming with {}~".format(type(transformer).__name__)
        if transformer != None:
            train_fs_x = transformer.transform(train_x)
            test_fs_x = transformer.transform(test_x)
        else:
            train_fs_x = train_x
            test_fs_x = test_x

        grid_search.fit(train_fs_x, train_y)
        grid_search.get_params()

        est = grid_search.best_estimator_
        est.fit(train_fs_x, train_y)
        pred_y = est.predict(test_fs_x)

        mse = mean_squared_error(test_y, pred_y)
        "----MSE: {}----".format(mse)
        if (mse < best_mse):
            best_mse = mse
            best_est = est
            best_transformer = transformer


~~~ Fitting and Testing DecisionTreeRegressor ~~~


'~Transforming with SelectFromModel~'

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None], 'criterion': ['mse', 'mae'], 'max_depth': [1000, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

{'cv': None,
 'error_score': 'raise',
 'estimator': DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
 'estimator__criterion': 'mse',
 'estimator__max_depth': None,
 'estimator__max_features': None,
 'estimator__max_leaf_nodes': None,
 'estimator__min_impurity_split': 1e-07,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__presort': False,
 'estimator__random_state': None,
 'estimator__splitter': 'best',
 'fit_params': {},
 'iid': True,
 'n_jobs': 1,
 'param_grid': {'criterion': ['mse', 'mae'],
  'max_depth': [1000, None],
  'max_features': [None]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': True,
 'scoring': None,
 'verbose': 0}

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

...

'~Transforming with SelectKBest~'

GridSearchCV(cv=None, error_score='raise',
       estimator=LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0),
       fit_params={}, iid=True, n_jobs=1, param_grid={'C': [1, 2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

{'cv': None,
 'error_score': 'raise',
 'estimator': LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
      intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
      random_state=None, tol=0.0001, verbose=0),
 'estimator__C': 1.0,
 'estimator__dual': True,
 'estimator__epsilon': 0.0,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1.0,
 'estimator__loss': 'epsilon_insensitive',
 'estimator__max_iter': 1000,
 'estimator__random_state': None,
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'fit_params': {},
 'iid': True,
 'n_jobs': 1,
 'param_grid': {'C': [1, 2, 4]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': True,
 'scoring': None,
 'verbose': 0}

LinearSVR(C=4, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

'----MSE: 0.228364529247----'

In [15]:
# now predict for everything using best
print("Best estimator is {}".format(type(best_est).__name__))

full_trans_x = vt.transform(full_x)
full_trans_x = best_transformer.transform(full_trans_x)
pred_y = best_est.predict(full_trans_x)
len(pred_y)

# set the variable to be tested to output
if test_var == 'gpa':
    pred_gpa_y = pred_y
elif test_var == 'materialHardship':
    pred_material_y = pred_y
elif test_var == 'grit':
    pred_grit_y = pred_y
elif test_var == 'eviction':
    pred_evict_y= pred_y
elif test_var == 'layoff':
    pred_layoff_y = pred_y
elif test_var == 'jobTraining':
    pred_job_y = pred_y

prediction_np = np.column_stack([
    np.arange(1,n,dtype=int), pred_gpa_y, pred_grit_y, pred_material_y, pred_evict_y, pred_layoff_y, pred_job_y])

prediction = pd.DataFrame(prediction_np,
                          columns=["challengeID", "gpa", "grit", "materialHardship", "eviction", "layoff", "jobTraining"])
prediction.head()
prediction.tail()

prediction.to_csv("prediction_out.csv", index=False)

Best estimator is ElasticNet


4242

,challengeID,gpa,grit,materialHardship,eviction,layoff,jobTraining
0,1.0,2.866,3.427,0.0596,0.0596,0.209,0.206871
1,2.0,2.866,3.427,0.0596,0.0596,0.209,0.185695
2,3.0,2.866,3.427,0.0596,0.0596,0.209,0.206871
3,4.0,2.866,3.427,0.0596,0.0596,0.209,0.236253
4,5.0,2.866,3.427,0.0596,0.0596,0.209,0.196283


,challengeID,gpa,grit,materialHardship,eviction,layoff,jobTraining
4237,4238.0,2.866,3.427,0.0596,0.0596,0.209,0.206871
4238,4239.0,2.866,3.427,0.0596,0.0596,0.209,0.185695
4239,4240.0,2.866,3.427,0.0596,0.0596,0.209,0.206871
4240,4241.0,2.866,3.427,0.0596,0.0596,0.209,0.314974
4241,4242.0,2.866,3.427,0.0596,0.0596,0.209,0.185695
